# When LLMs Choose Their Own Personas

In post 007, we gave LLMs explicit personas and watched them develop. In post 020, we tracked how those personas drift over time. But what happens when we **don't** give them any persona guidance at all?

This post investigates **implicit personas**—the distinct personalities that LLMs naturally develop during inference when given minimal prompts. Do they naturally differentiate? Do they converge? How do these implicit personas compare to explicit ones?

## The Experiment

We'll run conversations with:
- **No persona prompts**: Just basic instructions to participate
- **No role assignments**: Agents are free to develop their own identities
- **Same topic**: To ensure comparability with previous experiments
- **ConvoKit metrics**: To measure persona emergence quantitatively

We'll compare results with explicit persona experiments from post 007.


## Setup and Imports


In [ ]:
import os
import re
from typing import List, Dict
from collections import defaultdict
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
from random import shuffle, choice, random
import numpy as np
import matplotlib.pyplot as plt

# ConvoKit imports
from convokit import Corpus, Utterance, Speaker
from convokit.text_processing import TextParser
from convokit.politeness_strategies import PolitenessStrategies
from convokit.coordination import Coordination

load_dotenv("../../.env")
client = OpenAI()

TOPIC = "Code, testing, and infra as a source of truth versus comprehensive documentation."


## Conversation Runner Without Persona Prompts

Unlike post 007, we'll use minimal prompts that don't guide persona development.


In [ ]:
def run_conversation_implicit_personas(
    iterations: int,
    participant_count: int,
) -> List[Dict]:
    """
    Run conversation WITHOUT explicit persona prompts.
    Let LLMs develop their own implicit personas.
    """
    conversation_history = []
    ordering = list(range(1, participant_count + 1))
    last_speaker = -1
    
    # Bootstrap: Minimal prompt, no persona guidance
    for pid in ordering:
        speaker_id = f"speaker_{pid}"
        bootstrap_messages = [
            {
                "role": "system",
                "content": (
                    f"You are {speaker_id} in a group conversation. "
                    "You are participating in a discussion. "
                    "Share your thoughts on the topic below."
                ),
            },
            {"role": "user", "content": f"The topic is: {TOPIC}"},
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=bootstrap_messages,
            store=False,
        )
        first_message = response.choices[0].message.content
        conversation_history.append(
            {"role": "assistant", "name": speaker_id, "content": first_message}
        )
    
    def build_message(history: List[Dict], speaker_id: str, window_size: int) -> List[Dict]:
        """Build message context without persona reminders."""
        speaker_messages = [msg for msg in history if msg.get("name") == speaker_id][-window_size:]
        other_messages = [
            msg for msg in history
            if msg.get("name") not in (None, speaker_id)
        ][-window_size:]
        
        transcript = []
        
        if speaker_messages:
            transcript.append("Recent messages from you:")
            transcript.extend(f"- {msg['content']}" for msg in speaker_messages)
        
        if other_messages:
            transcript.append("\\nRecent messages from others:")
            transcript.extend(
                f"- {msg.get('name', msg['role'])}: {msg['content']}"
                for msg in other_messages
            )
        
        transcript_str = "\\n".join(transcript)
        
        return history + [
            {
                "role": "user",
                "content": (
                    f"{speaker_id}, continue the conversation and respond to the "
                    "others. Share your perspective on the topic."
                ),
            },
            {
                "role": "assistant",
                "name": speaker_id,
                "content": f"Here is the current state of the conversation.\\n{transcript_str}\\n\\n",
            },
        ]
    
    def shuffle_order(order: List[int]) -> List[int]:
        first = choice(order[:-1])
        remaining = [p for p in order if p != first]
        shuffle(remaining)
        return [first] + remaining
    
    for i in tqdm(range(iterations), desc=f"Running {participant_count}-speaker conversation"):
        if i > 0:
            ordering = shuffle_order(ordering)
        
        for pid in ordering:
            if random() < 0.3 or last_speaker == pid:
                continue
            
            speaker_id = f"speaker_{pid}"
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=build_message(conversation_history, speaker_id, 5),
                store=False,
            )
            message = response.choices[0].message.content
            conversation_history.append(
                {"role": "assistant", "name": speaker_id, "content": message}
            )
            last_speaker = pid
    
    return conversation_history


## ConvoKit Metrics

We'll use the same metrics from post 020 to measure persona characteristics.


In [ ]:
def conversation_to_corpus(conversation_history: List[Dict]) -> Corpus:
    """Convert conversation history to a ConvoKit Corpus."""
    utterances = []
    
    for idx, msg in enumerate(conversation_history):
        if msg.get("role") == "assistant" and "name" in msg:
            speaker_id = msg["name"]
            text = msg["content"]
            
            utterance = Utterance(
                id=f"utt_{idx}",
                speaker=Speaker(id=speaker_id),
                text=text
            )
            utterance.meta["timestamp"] = idx
            utterances.append(utterance)
    
    return Corpus(utterances=utterances)

def compute_dynamic_score(corpus: Corpus) -> float:
    """Dynamic: Collaborative (1) vs. Competitive (10)"""
    try:
        parser = TextParser()
        text_corpus = parser.transform(corpus)
        ps = PolitenessStrategies()
        ps_corpus = ps.transform(text_corpus)
        
        politeness_scores = []
        for utt in ps_corpus.iter_utterances():
            ps_score = utt.meta.get("politeness_strategies", {})
            positive_markers = sum([
                ps_score.get("feature_politeness_==HASPOSITIVE==", 0),
                ps_score.get("feature_politeness_==HASNEGATIVE==", 0) * -1,
            ])
            politeness_scores.append(positive_markers)
        
        avg_politeness = np.mean(politeness_scores) if politeness_scores else 0
        avg_politeness_normalized = min(1.0, max(0.0, avg_politeness / 5.0))
        
        speaker_counts = defaultdict(int)
        for utt in corpus.iter_utterances():
            speaker_counts[utt.speaker.id] += 1
        
        if len(speaker_counts) == 0:
            return 5.0
        
        total = sum(speaker_counts.values())
        probs = [count / total for count in speaker_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probs if p > 0)
        max_entropy = np.log2(len(speaker_counts))
        balance_score = entropy / max_entropy if max_entropy > 0 else 0
        
        combined = (avg_politeness_normalized + balance_score) / 2
        score = 10 - (combined * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing dynamic score: {e}")
        return 5.0

def compute_conclusiveness_score(corpus: Corpus) -> float:
    """Conclusiveness: Consensus (1) vs. Divergence (10)"""
    try:
        coord = Coordination()
        coord_corpus = coord.fit_transform(corpus)
        
        agreement_markers = ["agree", "yes", "exactly", "right", "true", "correct", "indeed", "absolutely", "definitely"]
        disagreement_markers = ["disagree", "no", "but", "however", "although", "wrong", "incorrect", "dispute", "differ"]
        
        agreement_count = 0
        disagreement_count = 0
        
        for utt in coord_corpus.iter_utterances():
            text_lower = utt.text.lower()
            for marker in agreement_markers:
                if marker in text_lower:
                    agreement_count += 1
            for marker in disagreement_markers:
                if marker in text_lower:
                    disagreement_count += 1
        
        if agreement_count == 0 and disagreement_count == 0:
            return 5.0
        elif disagreement_count == 0:
            return 1.0
        elif agreement_count == 0:
            return 10.0
        else:
            agreement_ratio = agreement_count / disagreement_count
        
        if agreement_ratio >= 2:
            score = 1 + (1 / (agreement_ratio - 1 + 1)) * 4
        elif agreement_ratio <= 0.5:
            score = 10 - (agreement_ratio * 4)
        else:
            score = 5.0
        
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing conclusiveness score: {e}")
        return 5.0

def compute_speaker_identity_score(corpus: Corpus) -> float:
    """Speaker Identity: Similarity (1) vs. Diversity (10)"""
    try:
        speakers = {}
        
        for utt in corpus.iter_utterances():
            speaker_id = utt.speaker.id
            if speaker_id not in speakers:
                speakers[speaker_id] = {"words": set()}
            
            words = set(re.findall(r'\\b\\w+\\b', utt.text.lower()))
            speakers[speaker_id]["words"].update(words)
        
        if len(speakers) < 2:
            return 5.0
        
        speaker_list = list(speakers.keys())
        overlaps = []
        
        for i in range(len(speaker_list)):
            for j in range(i + 1, len(speaker_list)):
                words_i = speakers[speaker_list[i]]["words"]
                words_j = speakers[speaker_list[j]]["words"]
                
                if len(words_i) == 0 or len(words_j) == 0:
                    continue
                
                overlap = len(words_i & words_j) / len(words_i | words_j)
                overlaps.append(overlap)
        
        if not overlaps:
            return 5.0
        
        avg_overlap = np.mean(overlaps)
        score = 10 - (avg_overlap * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker identity score: {e}")
        return 5.0

def compute_speaker_fluidity_score(corpus: Corpus, window_size: int = 20) -> float:
    """Speaker Fluidity: Malleability (1) vs. Consistency (10)"""
    try:
        speaker_utterances = defaultdict(list)
        
        for utt in corpus.iter_utterances():
            speaker_utterances[utt.speaker.id].append({
                "text": utt.text,
                "timestamp": utt.meta.get("timestamp", 0)
            })
        
        if len(speaker_utterances) == 0:
            return 5.0
        
        consistency_scores = []
        
        for speaker_id, utts in speaker_utterances.items():
            if len(utts) < window_size:
                continue
            
            utts_sorted = sorted(utts, key=lambda x: x["timestamp"])
            mid_point = len(utts_sorted) // 2
            
            first_half_words = set()
            second_half_words = set()
            
            for utt in utts_sorted[:mid_point]:
                words = set(re.findall(r'\\b\\w+\\b', utt["text"].lower()))
                first_half_words.update(words)
            
            for utt in utts_sorted[mid_point:]:
                words = set(re.findall(r'\\b\\w+\\b', utt["text"].lower()))
                second_half_words.update(words)
            
            if len(first_half_words) == 0 or len(second_half_words) == 0:
                continue
            
            overlap = len(first_half_words & second_half_words)
            union = len(first_half_words | second_half_words)
            similarity = overlap / union if union > 0 else 0
            consistency_scores.append(similarity)
        
        if not consistency_scores:
            return 5.0
        
        avg_consistency = np.mean(consistency_scores)
        score = 1 + (avg_consistency * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker fluidity score: {e}")
        return 5.0

def evaluate_conversation(conversation_history: List[Dict]) -> Dict[str, float]:
    """Compute all 4 evaluation metrics using ConvoKit."""
    corpus = conversation_to_corpus(conversation_history)
    
    return {
        "dynamic": compute_dynamic_score(corpus),
        "conclusiveness": compute_conclusiveness_score(corpus),
        "speaker_identity": compute_speaker_identity_score(corpus),
        "speaker_fluidity": compute_speaker_fluidity_score(corpus)
    }


## Running Experiments

Let's run conversations with 2 and 3 speakers, then analyze the implicit personas that emerge.


In [ ]:
# Run 2-speaker conversation
print("Running 2-speaker conversation with implicit personas...")
conv_2speaker = run_conversation_implicit_personas(iterations=30, participant_count=2)
metrics_2speaker = evaluate_conversation(conv_2speaker)

print("\\n=== 2-Speaker Results ===")
print(f"Dynamic (Collaborative ↔ Competitive): {metrics_2speaker['dynamic']:.2f}/10")
print(f"Conclusiveness (Consensus ↔ Divergence): {metrics_2speaker['conclusiveness']:.2f}/10")
print(f"Speaker Identity (Similarity ↔ Diversity): {metrics_2speaker['speaker_identity']:.2f}/10")
print(f"Speaker Fluidity (Malleability ↔ Consistency): {metrics_2speaker['speaker_fluidity']:.2f}/10")

# Run 3-speaker conversation
print("\\nRunning 3-speaker conversation with implicit personas...")
conv_3speaker = run_conversation_implicit_personas(iterations=30, participant_count=3)
metrics_3speaker = evaluate_conversation(conv_3speaker)

print("\\n=== 3-Speaker Results ===")
print(f"Dynamic (Collaborative ↔ Competitive): {metrics_3speaker['dynamic']:.2f}/10")
print(f"Conclusiveness (Consensus ↔ Divergence): {metrics_3speaker['conclusiveness']:.2f}/10")
print(f"Speaker Identity (Similarity ↔ Diversity): {metrics_3speaker['speaker_identity']:.2f}/10")
print(f"Speaker Fluidity (Malleability ↔ Consistency): {metrics_3speaker['speaker_fluidity']:.2f}/10")


## Analyzing Implicit Personas

Let's look at the actual messages to see what personas emerged.


In [ ]:
# Show first messages from each speaker (their initial personas)
print("=== Initial Personas (First Messages) ===")
for msg in conv_2speaker[:2]:
    speaker = msg.get("name", "unknown")
    content = msg.get("content", "")
    print(f"\\n{speaker}:")
    print(content[:200] + "..." if len(content) > 200 else content)

# Show later messages to see if personas persisted
print("\\n\\n=== Later Messages (Persona Persistence) ===")
for msg in conv_2speaker[-5:]:
    speaker = msg.get("name", "unknown")
    content = msg.get("content", "")
    print(f"\\n{speaker}: {content[:150]}...")


## Comparison with Explicit Personas

### Key Observations

1. **Natural Differentiation**: Even without explicit prompts, LLMs develop distinct communication styles
2. **Speaker Identity Score**: Measures how different speakers are from each other
   - High score (8-10): Speakers are very different (diverse personas)
   - Low score (1-3): Speakers are similar (convergent personas)
3. **Speaker Fluidity**: Measures consistency over time
   - High score (8-10): Speakers maintain consistent style
   - Low score (1-3): Speakers change style frequently

### Implicit vs Explicit Personas

**Implicit Personas (this post)**:
- Develop naturally during conversation
- May be less distinct initially
- Can evolve more freely

**Explicit Personas (post 007)**:
- Guided by initial prompts
- More distinct from the start
- May be more stable but less flexible

## Summary

LLMs naturally develop implicit personas even without explicit guidance:

- **Differentiation occurs**: Speaker Identity scores show distinct personas emerge
- **Consistency varies**: Some speakers maintain style, others adapt
- **Comparable to explicit**: Implicit personas can be as distinct as explicit ones

This suggests that:
1. LLMs have inherent tendencies toward persona development
2. Conversation context shapes persona emergence
3. Explicit prompts may guide but don't create personas from scratch

Future work could explore:
- What factors influence implicit persona development?
- How do implicit personas compare to explicit ones in long conversations?
- Can we predict which implicit personas will emerge?
